# Python Scripts testings and development

In [ ]:
import os
import glob
import torch
import open_clip # open_clip_torch
from PIL import Image
import faiss
from langchain_community.vectorstores import FAISS
from openai import AzureOpenAI
import numpy as np
from base64 import b64encode


import sys
import os

# Add the parent directory of 'other/' to sys.path
sys.path.append(os.path.abspath('..'))

from scripts.clip_model import CLIPImageEmbeddings

In [63]:

# -------------------------------
# 1️⃣ SETUP
# -------------------------------

client = AzureOpenAI(
    api_version="2024-02-01"
)

# # Load open_clip model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
# tokenizer = open_clip.get_tokenizer('ViT-B-32')

# model = model.to(device)


In [64]:
# -------------------------------
# 2️⃣ IMAGE EMBEDDING & INDEX
# -------------------------------

image_folder = "../images"
# image_paths = glob.glob(os.path.join(image_folder, "*.png"))
image_paths = os.listdir(image_folder)
clip_model = CLIPImageEmbeddings()
image_embeddings = {}
# image_ids = []
print(image_paths)
for img_path in image_paths:
    img_emb = clip_model.embed_image(os.path.join(image_folder,img_path))
    doc_id = img_path.split('.')[0]
    image_embeddings.update({doc_id:img_emb})
    print(doc_id,img_emb)
    '''
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        img_emb = model.encode_image(image) 
        img_emb /= img_emb.norm(dim=-1, keepdim=True)
        img_emb = img_emb.cpu().numpy()
    # image_embeddings.append(img_emb)
    ''' 
    # image_ids.append(img_path)
    

# image_embeddings = np.vstack(image_embeddings).astype('float32')



['79923dfb-0f72-49b0-ab05-578fe8eef824.png', '09a9f4be-8057-43d7-bdf2-a20404ab2165.png', '07393f28-9525-4a42-9bb5-2d153696739e.png']
79923dfb-0f72-49b0-ab05-578fe8eef824 [0.18251881003379822, -0.32610538601875305, -0.22274720668792725, -0.4841604232788086, -0.023906491696834564, -0.2721882462501526, 0.47721463441848755, 0.23225823044776917, 0.2279195785522461, -0.07989592850208282, 0.09799300134181976, 0.3387249708175659, 0.2280997931957245, 0.5066046714782715, 0.19521784782409668, 0.1484401673078537, 0.49362874031066895, 0.4825611710548401, -0.4426891505718231, -0.17349711060523987, 0.11929424107074738, 0.05512239784002304, 0.11747171729803085, 0.21381427347660065, -0.07943078130483627, 0.4164104163646698, -0.37203219532966614, 0.774694561958313, 0.17325608432292938, -0.17237895727157593, -0.03374110162258148, -0.004633799195289612, 0.30604487657546997, 0.4525325298309326, 0.31387659907341003, 0.14081579446792603, 0.3832182288169861, 0.5304779410362244, -0.5912972688674927, -1.8554878

In [45]:

# index = faiss.IndexFlatIP(image_embeddings.shape[1])

# print(list(image_embeddings.values())[0])

for key, img_emb in image_embeddings.items():
    doc_id = key
    vector_store = FAISS.from_embeddings(
        [(doc_id, img_emb)],
        embedding=clip_model
    )

vector_store.save_local("faiss_index")

# index.add(image_embeddings)

In [46]:

user_question = "what do you mean by scaled dot"

In [47]:
# -------------------------------
# 3️⃣ TEXT EMBEDDING
# -------------------------------


text_tokens = tokenizer([user_question]).to(device)
with torch.no_grad():
    text_emb = model.encode_text(text_tokens)
    text_emb /= text_emb.norm(dim=-1, keepdim=True)
    text_emb = text_emb.cpu().numpy().astype('float32')



In [48]:
# --------------------------------
# Loading Vector Store
# --------------------------------

index = FAISS.load_local('faiss_index', embeddings=clip_model, allow_dangerous_deserialization=True) 


In [49]:
# user_question_emb = model.encode_text(user_question)
user_question_emb = clip_model.embed_query(user_question)

In [60]:
# -------------------------------
# 4️⃣ SEARCH
# -------------------------------
user_question = "define mask(opt.) layer that comes after scale layer "
k = 1
retriver = index.as_retriever()
results = retriver.invoke(user_question)
# top_image_paths = [image_ids[i] for i in I[0]]

# print("Top matches:", top_image_paths)
results

[Document(id='6fb463b3-488d-48a8-b429-b2ffee96e76a', metadata={}, page_content='07393f28-9525-4a42-9bb5-2d153696739e')]

In [38]:
results[0].page_content

'07393f28-9525-4a42-9bb5-2d153696739e'

In [51]:
os.listdir('../images')

['79923dfb-0f72-49b0-ab05-578fe8eef824.png',
 '09a9f4be-8057-43d7-bdf2-a20404ab2165.png',
 '07393f28-9525-4a42-9bb5-2d153696739e.png']

In [7]:
# -------------------------------
# 5️⃣ AZURE GPT-4o-mini
# -------------------------------

context = f"The user asked: '{user_question}'. Relevant images: {', '.join(top_image_paths)}."

img_path = top_image_paths[0] # "images/07393f28-9525-4a42-9bb5-2d153696739e.png"

# fetching image data
with open(img_path, "rb") as f:
    image_b64 = b64encode(f.read()).decode("utf-8")

data_url = f"data:image/png;base64,{image_b64}"                     # creating data url variable


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are an expert vision assistant. Answer the query from the given image ONLY, if no answer will found then say you don't know."
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"user query: {user_question}"},
                {"type": "image_url", "image_url": {"url": data_url}}
            ]
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0
)

print("\n💡 LLM Response:")
print(response.choices[0].message.content)



💡 LLM Response:
I don't know.
